# Coffee Shop Sales Dashboard
Created by ***Péter Szilvási***

---
### 1. Dataset description
Data is about transaction records of a fictions cofee shop operating in NYC.<br>
(Source: [Maven Analytics](https://mavenanalytics.io/data-playground?page=2&pageSize=5))<br>
Dataset variables include:
- transaction date
- timestamp 
- location
- product-level details
    - product category
    - quantities sold
    - unit per price
---
### 2. Workflow
1. Importing libraries and data from my public [GitHub repository](https://github.com/szilvasipeter2000/Data_Visualization)
2. Correcting Column data types
3. Creating new columns to fit our purposes
4. Creating python variables to help with filtering
5. Defining App and creating visualizations
---
### 3. Output
The Dashboard aims to visualize daily trends and sales by month,location and product category.<br>
All 3 graphs are to be interactive with filters.<br>
It is meant to be looked at from separate window: http://127.0.0.1:8050/

In [1]:
# import libraries
import pandas as pd
from dash import Dash 
from dash import html, dcc 
from dash import callback, Output, Input
import plotly.express as px

# Load the csv file into a pandas DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/szilvasipeter2000/Data_Visualization/main/coffee_sales-dashboard/data/coffee_sales.csv',sep=';')


############################################################################
# Fix data types
## datetime for date and time
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df['transaction_time'] = pd.to_datetime(df['transaction_time'])
## int for transaction_qty
df['transaction_qty'] = df['transaction_qty'].astype('int')
## float for unit_price, but replace , with . first
df['unit_price'] = df['unit_price'].replace(',','.', regex=True)
df['unit_price'] = df['unit_price'].astype('float')

############################################################################
# Create more columns to help graphs
## create sales column. Quantity * price
df['sales'] = df['transaction_qty'] * df['unit_price']
## create month column. We want to look at sales data by month
df['month'] = df['transaction_date'].dt.month.map({1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June',7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'})
## create year column is not needed, data is only for 2023
## create weekend column 0 = weekday, 1 = weekend. Trends might differ on weekends and weekdays
df['weekend'] = df['transaction_date'].dt.weekday.map({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1})

############################################################################
# Create category variables for filters:
# 1. product categories
categories = df.product_category.unique()
categories.sort()
# 2. stres/locations
stores = df.store_location.unique()
stores.sort()

############################################################################
# Creating the Dashboard itself
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Coffee Shop Sales', style={'textAlign': 'center', 'fontSize': 36, 'fontFamily': 'Arial, sans-serif'}),
    
    # First Row for Filters
    html.Div([
        html.Label('Select Store Location: '),
        dcc.Checklist(id='checklist1', options=[{'label': store, 'value': store} for store in stores], # create filter for each store, used Checklist instead of RadioItems to allow multiple selections 
                      value=['Astoria',"Hell's Kitchen"], inline=True, style={'margin-right': '20px'}), # default is Astoria and Hell's Kitchen
        html.Label('Select Product Category: '),
        dcc.Dropdown(id='dropdown1', options=categories, value='Coffee', style={'width': '35%'}), # create dropdown filter for product categories
    ], style={'margin-bottom': '20px'}),
    
    # Second Row For Histogram1 (Daily Trends)
    html.Div(dcc.Graph(id='graph1'), style={'display': 'inline-block', 'width': '100%'}), # I will add facets for weekends and weekdays so this needs to be 100% wide
    
    # Third Ror for Histogram2 (Monthly Sales) and Pie Chart (Distribution among locations). these two will be side by side
    html.Div([
        html.Div(dcc.Graph(id='graph2'), style={'display': 'inline-block', 'width': '50%'}), # sales over months 50% so it goes better with the chart above it in the first row
        html.Div(dcc.Graph(id='graph3'), style={'display': 'inline-block', 'width': '40%'}), # with 40% width the pie chart better fits the chart above, it's more to the middle 
    ]),
])

@app.callback(
    Output('graph1', 'figure'), # output for histogram1
    Output('graph2', 'figure'), # output for histogram2
    Output('graph3', 'figure'), # output for pie chart
    Input('dropdown1', 'value'), # input for product category filter
    Input('checklist1', 'value') # input for store location filter
)
# create function to update graphs when filters are changed
def update1(dropdown1, checklist1): # dropdown1 is the product category filter, radio1 is the store location filter

     
    df_dash = df.query('product_category == @dropdown1 and store_location in @checklist1') # filter the DataFrame based on the filters

    # For the first Histogram I want to see human behavior in the data. What time during the day people buy coffee,tea etc. This is regardless of store location, so using barmode is not necessary.
    # I use histograms because bar graph couldn't show all the data. Also this way I can use bins.
    histogram1 = px.histogram(df_dash, # create histogram for daily trends
                            x='transaction_time', 
                            y='transaction_qty', 
                            color='store_location', # color by store location, so trends are seen.  
                            nbins=150, #with more bins the graph is more detailed
                            template = 'simple_white', # remove grey background
                            facet_col='weekend', # create facets for weekends and weekdays. Trends might differ on weekends and weekdays
                            labels = {'store_location' : 'Store Location', 'transaction_time' : ''} # rename legend label and 'remove' x labels for 2 facets
                            )
    
    histogram1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1].replace('0', 'Weekdays').replace('1', 'Weekends')))  # rename facet labels from 0 and 1 to Weekdays and Weekends
    
    histogram1.update_layout(title=dict(text='<b>Daily Trends for category:</b> ' + str(dropdown1),font=dict(size=20)), # add title, add bold formatting and make the title interactive
                            title_x = 0.5, # move title to the middle
                            xaxis_title = '', # remove x axis title, don't remove the whole x axis as i want to keep the time labels
                            yaxis = dict(visible=False), # remove y axis, data labels are not needed. trends are seen by the height of the bars and hovering over them is enough if we wish to see the exact values 
                            legend=dict(x=1, y=1,font=dict(size=16)) # move legend to the top right corner and increase font size as this will be the only legend for all 3 graphs
                            )
    # Histogram2 is for monthly sales.
    histogram2 = px.histogram(df_dash,
                              x='month',
                              y='sales',
                              barmode='group', # barmode to see the total sales for each month by store location
                              color='store_location', #  also using colors to help this.
                              template = 'simple_white' # remove grey background
                              )
    
    histogram2.update_layout(bargap=0.1,
                             title=dict(text='<b>Sales over months for category: </b>' + str(dropdown1),font=dict(size=20)), # add title, add bold formatting and make the title interactive
                             title_x = 0.5, # move title to the middle
                             xaxis_title = '', # remove x axis title, don't remove the whole x axis as i want to keep the month labels
                             yaxis_title = '', # remove y axis title
                             yaxis = dict(visible=False), # remove y axis, I will use data labels
                             showlegend=False # remove legend, as it's the same as the first histogram
                             )
    
    histogram2.update_traces(texttemplate = '%{value:.3s}', textposition='outside') # add data labels outside to the bars
    
    # Pie chart for distribution among locations
    pie1 = px.pie(df_dash,
                    names='store_location',
                    values='sales',
                    labels={'store_location': 'Store Location'}, # rename legend label from original column name to 'Store Location'
                    template = 'simple_white', # to match the other graphs' theme
                    )
    pie1.update_traces(textinfo='label,value+percent', # add data labels to know which location it is with value and percentage of total
                   texttemplate='%{label}:<br> %{value:.3s} (%{percent})', # add formatting to data labels, also add line break
                   textposition='outside' # move data labels outside the pie chart
                   )

    pie1.update_layout(title = dict(text='<b>Distribution among Locations: </b>'+ str(checklist1).replace('[','').replace(']','').replace("'","").replace('"',""),font=dict(size=20)), # add title, add bold formatting and make the title interactive
                       title_x = 0.5, # move title to the middle
                       showlegend=False # remove legend, as it's the same as the first histogram
                       )

    return histogram1, histogram2, pie1 # return the 3 graphs to finish the callback function

app.run_server(debug=True) # run the app
